In [1]:
import tensorflow as tf
import numpy as np

from keras.layers import Input
from tensorflow import Tensor
from keras import Model
from keras.layers import Dense
from tensorflow.nn import softmax
from compute import create_vector_probability_attention
from Embedding import Embedding_Layer
from Attention import Attention_Layer
from Multi_Head_Attention import Multi_Head_Attention_Layer
from keras.layers import Normalization
from Encoding import Encoder_Layer


2022-10-02 12:01:15.271424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-02 12:01:15.271449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
input_embedding = [[
  "Salut", "comment", "ca", "va", "?"
]]


output_embedding = [[
    "<START>", "Hi", "how", "are", "you", "?"
]]

def get_vocabulary(sequences):

  token_to_info = {}

  for sequence in sequences:
    for word in sequence:
      if word not in token_to_info:
        token_to_info[word] = len(token_to_info)
  
  return token_to_info

input_voc = get_vocabulary(input_embedding)
output_voc = get_vocabulary(output_embedding)

input_voc["<START>"] = len(input_voc)
input_voc["<END>"] = len(input_voc)
input_voc["<PAD>"] = len(input_voc)

output_voc["<END>"] = len(output_voc)
output_voc["<PAD>"] = len(output_voc)

print(input_voc)
print(output_voc)

def sequences_to_int(sequences, voc):
  for sequence in sequences:
    print(sequence)
    for s, word in enumerate(sequence):
      sequence[s] = voc[word]
  return np.array(sequences)

input_seq = sequences_to_int(input_embedding, input_voc)
output_seq = sequences_to_int(output_embedding, output_voc)

print("input_seq", input_seq)
print("output_seq", output_seq)

{'Salut': 0, 'comment': 1, 'ca': 2, 'va': 3, '?': 4, '<START>': 5, '<END>': 6, '<PAD>': 7}
{'<START>': 0, 'Hi': 1, 'how': 2, 'are': 3, 'you': 4, '?': 5, '<END>': 6, '<PAD>': 7}
['Salut', 'comment', 'ca', 'va', '?']
['<START>', 'Hi', 'how', 'are', 'you', '?']
input_seq [[0 1 2 3 4]]
output_seq [[0 1 2 3 4 5]]


In [3]:
class  Decoding_Layer(tf.keras.layers.Layer):
    
    def __init__(self, dim : int, nbr_heads : int, mask_size : int,  **kwargs):
        self.dim = dim
        self.nbr_heads = nbr_heads
        self.mask_size = mask_size
        super(**kwargs).__init__()
       
    def build(self, input_shape):
        # self.masked_multi_head_attention = Masked_Multi_Head_Attention_Layer(self.dim, self.nbr_heads, self.mask_size)
        # self.normalization = Normalization()
        # self.dense = Dense(256)
        super().build(input_shape)
    
    def call(self, x):
        
        encodeur, output_embedding = x
        # attention = self.masked_multi_head_attention(x)
        # attention_normalize = self.normalization(attention + x)
        # dense = self.dense(attention_normalize)
        # output = self.normalization(dense + attention_normalize)
        
        return (x) 
    
class Decoder_Layer(tf.keras.layers.Layer):
    
    def __init__(self, nb_decoder : int, dim : int, nbr_heads : int, mask_size : int,  **kwargs):
        self.nb_decoder = nb_decoder
        self.dim = dim
        self.nbr_heads = nbr_heads
        self.mask_size = mask_size
        super(**kwargs).__init__()
        
    def build(self, input_shape):
        
        self.decoder_layer_list = []

        for _ in range(self.nb_decoder):
            decoder_layer = Decoding_Layer(self.dim, self.nbr_heads, self.mask_size)
            self.decoder_layer_list.append(decoder_layer)
    
    def call(self, x):
        
        encoder, output_embedding = x
        
        decoder_output = output_embedding
        
        for decoder_layer in self.decoder_layer_list:
            decoder_output = decoder_layer((encoder, decoder_output))
        return (x);

In [6]:

NBR_TOKEN = 5
INPUT_SIZE = 5
OUTPUT_SIZE = 6

def build_Model(coding_block : int, dim : int, nbr_heads : int, mask_size : int):
    
    input_token = Input(shape=(INPUT_SIZE))
    output_token = Input(shape=(OUTPUT_SIZE))
    
    input_embedding = Embedding_Layer(INPUT_SIZE)(input_token)
    output_embedding = Embedding_Layer(OUTPUT_SIZE)(output_token)
    
    encoder = Encoder_Layer(coding_block, dim, nbr_heads)(input_embedding)
    
    decoder = Decoder_Layer(coding_block, dim, nbr_heads, mask_size)((encoder, output_embedding))

    model = Model([input_token, output_token], decoder)
    model.summary()
    return (model)

In [8]:
model = build_Model(coding_block=6, dim=256, nbr_heads=8, mask_size=6)
output = model((input_seq, output_seq))

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 embedding__layer_4 (Embedding_  (None, 5, 256)      1280        ['input_5[0][0]']                
 Layer)                                                                                           
                                                                                                  
 input_6 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 encoder__layer_2 (Encoder_Laye  (None, 5, 256)      1582086     ['embedding__layer_4[0][0]'

In [27]:
output

<tf.Tensor: shape=(1, 5, 256), dtype=float32, numpy=
array([[[ 1.4503533 , -0.3028434 ,  0.40352502, ..., -0.77106094,
          1.1617829 , -0.6936591 ],
        [ 1.246748  ,  0.2138382 ,  0.46494538, ..., -0.9506548 ,
          1.1282066 , -0.61845905],
        [ 1.498886  ,  0.28181407,  0.37907103, ..., -0.81980187,
          0.9780137 , -0.98988265],
        [ 0.9374498 ,  0.04110647,  0.517773  , ..., -0.75270104,
          1.2630426 , -1.001419  ],
        [ 1.3170918 ,  0.19625492,  0.639094  , ..., -0.43390927,
          0.8181146 , -0.38335136]]], dtype=float32)>